# List context: fast row-wise operations

The list context allows you to apply any expression over elements of the list. This enables row-wise computation.

Polars expressions work on columns which must have a homogeneous type. But rows have no such guarantee in Polars `DataFrame`s. However, the `pl.List` data type does enforce a homogenous type across a row. 

In [2]:
import polars as pl
from polars import col
import pandas as pd

In [3]:
grades = pl.DataFrame(
    {
        "student": ["bas", "laura", "tim", "jenny"],
        "arithmetic": [10, 5, 6, 8],
        "biology": [4, 6, 2, 7],
        "geography": [8, 4, 9, 7],
    }
)
grades


student,arithmetic,biology,geography
str,i64,i64,i64
"""bas""",10,4,8
"""laura""",5,6,4
"""tim""",6,2,9
"""jenny""",8,7,7


The following concatenates all columns into a list, excluding the student names:

In [5]:
grades.select([
    col('student'),
    pl.concat_list(pl.all().exclude('student').alias("all_grades"))
])

student,all_grades
str,list[i64]
"""bas""","[10, 4, 8]"
"""laura""","[5, 6, 4]"
"""tim""","[6, 2, 9]"
"""jenny""","[8, 7, 7]"


## Running polars expressions on list elements

Compute the rank percentage

`pl.element()` allows you to select the element of a list. `arr.eval()` allows you to run expressions on a list.

In [11]:
rank_pct = pl.element().rank(reverse=True)/col("").count()

In [12]:
grades.with_column(
    pl.concat_list(pl.all().exclude("student")).alias("all_grades")
).select([
    pl.all().exclude("all_grades"),
    col("all_grades").arr.eval(rank_pct, parallel=True).alias("grades_rank")
])

student,arithmetic,biology,geography,grades_rank
str,i64,i64,i64,list[f64]
"""bas""",10,4,8,"[0.333333, 1.0, 0.666667]"
"""laura""",5,6,4,"[0.666667, 0.333333, 1.0]"
"""tim""",6,2,9,"[0.666667, 1.0, 0.333333]"
"""jenny""",8,7,7,"[0.333333, 0.833333, 0.833333]"
